<a href="https://colab.research.google.com/github/Matheshmj/Fake_review_classification_and_topic_modelling/blob/main/Transformer_pre_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load data
data= pd.read_csv('/content/Downloads_cleaned_review_data.csv')
data

,Unnamed: 0,category,rating,label,text,Sentiment,tokens,joined_text
0,0,Home_and_Kitchen_5,5.0,CG,love this well made sturdy and very comfortab...,Positive,"['love', 'well', 'made', 'sturdy', 'comfortabl...",love well made sturdy comfortable love itvery ...
1,1,Home_and_Kitchen_5,5.0,CG,love it a great upgrade from the original ive...,Positive,"['love', 'great', 'upgrade', 'original', 'ive'...",love great upgrade original ive mine couple year
2,2,Home_and_Kitchen_5,5.0,CG,this pillow saved my back i love the look and ...,Positive,"['pillow', 'saved', 'back', 'love', 'look', 'f...",pillow saved back love look feel pillow
3,3,Home_and_Kitchen_5,1.0,CG,missing information on how to use it but it is...,Negative,"['missing', 'information', 'use', 'great', 'pr...",missing information use great product price
4,4,Home_and_Kitchen_5,5.0,CG,very nice set good quality we have had the set...,Positive,"['nice', 'set', 'good', 'quality', 'set', 'two...",nice set good quality set two month
...,...,...,...,...,...,...,...,...
40427,40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,i had read some reviews saying that this bra r...,Positive,"['read', 'review', 'saying', 'bra', 'ran', 'sm...",read review saying bra ran small ordered two b...
40428,40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,i wasnt sure exactly what it would be it is a ...,Positive,"['wasnt', 'sure', 'exactly', 'would', 'little'...",wasnt sure exactly would little large small si...
40429,40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,you can wear the hood by itself wear it with t...,Negative,"['wear', 'hood', 'wear', 'hood', 'wear', 'jack...",wear hood wear hood wear jacket without hood s...
40430,40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,i liked nothing about this dress the only reas...,Negative,"['liked', 'nothing', 'dress', 'reason', 'gave'...",liked nothing dress reason gave star ordered s...


In [ ]:
pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
from datasets import Dataset
import os

# Disable W&B Logging
os.environ["WANDB_DISABLED"] = "true"
data = data[['joined_text', 'label']]

data['label_encoded'] = data['label'].apply(lambda x: 0 if x == 'CG' else 1)  # Encode labels (binary classification)

X = data['joined_text']
y = data['label_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert problematic floats in 'text' column to strings
X_train = X_train.astype(str)
X_test = X_test.astype(str) # Apply the same to X_test for consistency

# Convert to Hugging Face Dataset
train_data = Dataset.from_dict({'text': X_train.tolist(), 'label': y_train.tolist()})
test_data = Dataset.from_dict({'text': X_test.tolist(), 'label': y_test.tolist()})

# Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=64)

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

# Format dataset for PyTorch
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load Pre-trained Model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Define Metrics
def compute_metrics(pred):
    predictions = np.argmax(pred.predictions, axis=1)
    labels = pred.label_ids
    accuracy = accuracy_score(labels, predictions)
    report = classification_report(labels, predictions, target_names=['CG', 'OR'], output_dict=True)
    return {"accuracy": accuracy, "classification_report": report}

# Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics
)

# Train and Evaluate
trainer.train()

# Evaluate on Test Set
results = trainer.evaluate()
print(f"\nTest Accuracy: {results['eval_accuracy']:.4f}")

# Detailed Report
print("\nDetailed Classification Report:")
print(results['eval_classification_report'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy,Classification Report
1,0.217900,0.210065,0.908248,"{'CG': {'precision': 0.8777111213659436, 'recall': 0.9472111553784861, 'f1-score': 0.9111377245508983, 'support': 4016.0}, 'OR': {'precision': 0.9435118571809219, 'recall': 0.8698108572832228, 'f1-score': 0.9051635991820041, 'support': 4071.0}, 'accuracy': 0.9082478051193273, 'macro avg': {'precision': 0.9106114892734327, 'recall': 0.9085110063308545, 'f1-score': 0.9081506618664512, 'support': 8087.0}, 'weighted avg': {'precision': 0.9108352459489505, 'recall': 0.9082478051193273, 'f1-score': 0.9081303467375226, 'support': 8087.0}}"
2,0.174300,0.200846,0.921974,"{'CG': {'precision': 0.8933302347199629, 'recall': 0.9571713147410359, 'f1-score': 0.9241495372039908, 'support': 4016.0}, 'OR': {'precision': 0.9545454545454546, 'recall': 0.8872512896094326, 'f1-score': 0.9196690006365372, 'support': 4071.0}, 'accuracy': 0.9219735377766787, 'macro avg': {'precision': 0.9239378446327087, 'recall': 0.9222113021752343, 'f1-score': 0.921909268920264, 'support': 8087.0}, 'weighted avg': {'precision': 0.924146008172365, 'recall': 0.9219735377766787, 'f1-score': 0.9218940327689589, 'support': 8087.0}}"
3,0.076400,0.212318,0.930258,"{'CG': {'precision': 0.9157032755298651, 'recall': 0.9467131474103586, 'f1-score': 0.9309500489715965, 'support': 4016.0}, 'OR': {'precision': 0.9456162642947904, 'recall': 0.9140260378285433, 'f1-score': 0.9295528353734699, 'support': 4071.0}, 'accuracy': 0.9302584394707555, 'macro avg': {'precision': 0.9306597699123278, 'recall': 0.930369592619451, 'f1-score': 0.9302514421725332, 'support': 8087.0}, 'weighted avg': {'precision': 0.9307614896095004, 'recall': 0.9302584394707555, 'f1-score': 0.9302466909206539, 'support': 8087.0}}"


Trainer is attempting to log a value of "{'CG': {'precision': 0.8777111213659436, 'recall': 0.9472111553784861, 'f1-score': 0.9111377245508983, 'support': 4016.0}, 'OR': {'precision': 0.9435118571809219, 'recall': 0.8698108572832228, 'f1-score': 0.9051635991820041, 'support': 4071.0}, 'accuracy': 0.9082478051193273, 'macro avg': {'precision': 0.9106114892734327, 'recall': 0.9085110063308545, 'f1-score': 0.9081506618664512, 'support': 8087.0}, 'weighted avg': {'precision': 0.9108352459489505, 'recall': 0.9082478051193273, 'f1-score': 0.9081303467375226, 'support': 8087.0}}" of type <class 'dict'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'CG': {'precision': 0.8933302347199629, 'recall': 0.9571713147410359, 'f1-score': 0.9241495372039908, 'support': 4016.0}, 'OR': {'precision': 0.9545454545454546, 'recall': 0.8872512896094326, 'f1-score': 0.919

Trainer is attempting to log a value of "{'CG': {'precision': 0.9157032755298651, 'recall': 0.9467131474103586, 'f1-score': 0.9309500489715965, 'support': 4016.0}, 'OR': {'precision': 0.9456162642947904, 'recall': 0.9140260378285433, 'f1-score': 0.9295528353734699, 'support': 4071.0}, 'accuracy': 0.9302584394707555, 'macro avg': {'precision': 0.9306597699123278, 'recall': 0.930369592619451, 'f1-score': 0.9302514421725332, 'support': 8087.0}, 'weighted avg': {'precision': 0.9307614896095004, 'recall': 0.9302584394707555, 'f1-score': 0.9302466909206539, 'support': 8087.0}}" of type <class 'dict'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.



Test Accuracy: 0.9303

Detailed Classification Report:
{'CG': {'precision': 0.9157032755298651, 'recall': 0.9467131474103586, 'f1-score': 0.9309500489715965, 'support': 4016.0}, 'OR': {'precision': 0.9456162642947904, 'recall': 0.9140260378285433, 'f1-score': 0.9295528353734699, 'support': 4071.0}, 'accuracy': 0.9302584394707555, 'macro avg': {'precision': 0.9306597699123278, 'recall': 0.930369592619451, 'f1-score': 0.9302514421725332, 'support': 8087.0}, 'weighted avg': {'precision': 0.9307614896095004, 'recall': 0.9302584394707555, 'f1-score': 0.9302466909206539, 'support': 8087.0}}
